# SPATIAL.ipynb 
# Geometric Data Tools

---
## Overview
- centroids.py
- geometry_optimize.py
- tract_joins.py

# 1 - centroids.py

This file is responsible for computing tract centroids.

# 1.1 - Imports

This section covers the necessary imports.

Annotations/type hints are imported for clarity. GeoPandas and Pandas are imported for data management.

In [ ]:
from __future__ import annotations
import geopandas as gpd
import pandas as pd

# 1.2 - Tract Centroids

This function takes tract polygons, computes each tract's center point, extracts latitude and longitude, and returns a dataframe containing GEOIDs and latitude/longitude. The function accepts a GeoDataFrame `tracts`, which are the census tracts ingested by fetch_census_tracts.

Line-by-line breakdown:
- Create a safe copy of `tracts`.
- Compute centroid (c.geometry.centroid returns a GeoSeries of Point objects, where each Point represents the center of its polygon). Store in a new column `centroid` in the safe copy `c`.
- Construct a Pandas dataframe with columns `GEOID`, `lat`, and `lon`, filled using the GEOID valus from `c`, the `.y` coordinate from `c["centroid"]`, and the `.x` coordinate from `c["centroid"]`, respectively. Return this dataframe.

NOTE: Why not a GeoDataFrame? In this case, a regular dataframe is easier to join with ACS data for building the layers in this project. It also feeds into Folium easier.

In [ ]:
def tract_centroids(tracts: gpd.GeoDataFrame) -> pd.DataFrame:
    c = tracts.copy()
    c["centroid"] = c.geometry.centroid
    return pd.DataFrame({"GEOID": c["GEOID"], "lat": c["centroid"].y, "lon": c["centroid"].x})


# 2 - geometry_optimize.py

This file is responsible for cleaning and normalizing geometry data to optimize the application's performance.

# 2.1 - Imports

This section covers the necessary imports.

Annotations/type hints are imported for clarity. GeoPandas is used for data management.

In [ ]:
from __future__ import annotations
import geopandas as gpd

# 2.2 - Simplify Polygons

This function is responsible for taking a complicated GeoDataFrame with polygon geometries and reduces geometric complexity. The function accepts a GeoDataFrame `gdf` (input polygon data) and a float `tolerance` (simplification sensitivity, higher value indicates fewer points). The return object is a simplified version of the input GeoDataFrame.

Line-by-line breakdown:
- Create a safe copy of the input data.
- `.simplify` reduces the number of vertices in each polygon, passing in the `tolerance` argument. By setting `preserve_topology` to true, the function prevents self-intersections and polygon collapse.
- `.buffer(0)` cleans geometry and rebuilds polygon boundaries to resolve reoccuring renerding failures from earlier versions of this project. The simplified and buffered dataframe is then returned.


NOTE 1: This function is CRUCIAL for allowing the application to run. Otherwise, computation times are brutal and Streamlit tends to crash.

NOTE 2: `.simplify` uses a Douglas-Puecker-style algorithm, which is fast and gets the job done. To improve visual quality, one might try Topology-Preserving Simplification, which is a bit slower but can still function well in this application. An earlier version of the project used Visvalingam-Whyatt algorithm, which was too slow and made for a frustraing user experience.

In [ ]:
def simplify_polygons(gdf: gpd.GeoDataFrame, tolerance: float = 0.001) -> gpd.GeoDataFrame:
    out = gdf.copy()
    out["geometry"] = out["geometry"].simplify(tolerance=tolerance, preserve_topology=True)
    out["geometry"] = out["geometry"].buffer(0)  # fix invalids
    return out

# 2.3 - Drop Large Columns

This function is responsible for reducing memory size to keep the application from running slowly. The function accepts a GeoDataFrame `gdf` (geometric data) and a list of strings `keep` (columns to keep in the geometric data). The return object is the input data with all "non-keep" columns dropped.

Line-by-line breakdown:
- Create a list of columns using `keep`, and ensure that the columns actually exist in the input data to avoid `KeyError`.
- Ensure that geometry data is kept in the dataframe.
- Return the modified GeoDataFrame.

In [ ]:
def drop_large_columns(gdf: gpd.GeoDataFrame, keep: list[str]) -> gpd.GeoDataFrame:
    cols = [c for c in keep if c in gdf.columns]
    if "geometry" not in cols:
        cols.append("geometry")
    return gdf[cols]

# 3 - tract_joins.py

This file hosts the join logic necessary for combining point data with tract data.

# 3.1 - Imports

This section covers the necessary imports.

Annotations/type hints are imported for clarity. GeoPandas and Pandas are used for data management.

In [ ]:
from __future__ import annotations
import geopandas as gpd
import pandas as pd

# 3.2 - Points-to-GDF

This function is responsible for converting a dataframe with latitude/longitude columns into a GeoDataFrame of points. The function accepts a dataframe `df` (the lat/lon dataframe) and a string `crs` (defaults to EPSG:4326 coordinate reference). The return is a GeoDataFrame.

`df_copy` is a safe copy of the input data. `.points_from_xy()` converts coordinate columns into Shapely Point objects, and returns a GeoSeries of Point(lon, lat). `crs` is the coordinate reference used (see ingest.ipynb for more information on CRS). The function passes these parameters into a GeoDataFrame constructor and returns it.

In [ ]:
def points_to_gdf(df: pd.DataFrame, crs: str = "EPSG:4326") -> gpd.GeoDataFrame:
    return gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs=crs,
    )

# 3.3 - Spatial Join Points to Tracts

This function assigns census tract GEOIDs to each point. The function accepts two GeoDataFrames `points_gdf` (the output of `points_to_gdf()`) and `tracts_gdf` (tract data). The return object is a joined GeoDataFrame.

Line-by-line breakdown:
- Check to see if the CRS specs between the input dataframes are the same. If not, convert the `points_gdf` CRS to the same one as `tracts_gdf`.
- Spatial join, where the points table is the left table, and the right table is the tracts table with only `GEOID` and `geomtetry` columns. The join is a left join, so all points are kept and points outside of tracts have a NaN GEOID. The `predicate=within` parameter ensures that points must lie inside polygons.
- `index_right` is dropped to prevent confusion.
- Return the joined GeoDataFrame.

In [ ]:
def spatial_join_points_to_tracts(points_gdf: gpd.GeoDataFrame, tracts_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    if points_gdf.crs != tracts_gdf.crs:
        points_gdf = points_gdf.to_crs(tracts_gdf.crs)
    joined = gpd.sjoin(points_gdf, tracts_gdf[["GEOID", "geometry"]], how="left", predicate="within")
    if "index_right" in joined.columns:
        joined = joined.drop(columns=["index_right"])
    return joined